In [1]:
import pandas as pd
import numpy as np

In [65]:
train = pd.read_csv("./data/train.csv")

In [ ]:
## Manejar columnas duplicadas: F_TIENEINTERNET y F_TIENEINTERNET_1
# ----------------------------
# Si existe la versión ".1", comprobar si son iguales; si es así, borrar una
if 'F_TIENEINTERNET.1' in train.columns:
    # comparar manteniendo NaNs
    eq_count = ((train['F_TIENEINTERNET'] == train['F_TIENEINTERNET.1']) | 
                (train['F_TIENEINTERNET'].isna() & train['F_TIENEINTERNET.1'].isna())).sum()
    if eq_count == len(train):
        train = train.drop(columns=['F_TIENEINTERNET.1'])
        print(f"Comparación de F_TIENEINTERNET y F_TIENEINTERNET.1: {eq_count} iguales de {len(train)} registros.") 
    else:
        # si tienen discrepancias, conservar la que consideres más fiable o reconciliar
        pass
    

## Valor de matricula

# Mapear a valores numéricos representativos (por ejemplo, rango medio o escala 1–8)

map_valor = {
    'No pagó matrícula': 0,
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7
}
train['E_VALORMATRICULAUNIVERSIDAD'] = train['E_VALORMATRICULAUNIVERSIDAD'].map(map_valor)

## educación padre y madre

# Mapear a escala 0–7 (por orden educativo), considerar imputar “No sabe/No aplica” con mediana o categoría baja

nivel_edu = {
  'Ninguno':0,
  'Primaria incompleta':1,
  'Primaria completa':2,
  'Secundaria (Bachillerato) incompleta':3,
  'Secundaria (Bachillerato) completa':4,
  'Técnica o tecnológica incompleta':5,
  'Técnica o tecnológica completa':6,
  'Educación profesional incompleta':7,
  'Educación profesional completa':8,
  'Postgrado':9,
  'No sabe':np.nan,
  'No Aplica':np.nan
}
train['F_EDUCACIONPADRE'] = train['F_EDUCACIONPADRE'].map(nivel_edu)
train['F_EDUCACIONMADRE'] = train['F_EDUCACIONMADRE'].map(nivel_edu)


## Rendimiento global

# Variable categórica ordinal

rend_map = {'bajo':0, 'medio-bajo':1, 'medio-alto':2, 'alto':3}
train['RENDIMIENTO_GLOBAL'] = train['RENDIMIENTO_GLOBAL'].map(rend_map)


## Mapeos para variables binarias

def map_binary(col):
    return (col.astype(str).str.strip().str.upper()
            .replace({'S':'SI','Y':'SI','YES':'SI','TRUE':'SI','1':'SI',
                      'N':'NO','NO':'NO','FALSE':'NO','0':'NO',
                      'NAN':'nan'}))

binary_cols = ['F_TIENEINTERNET','F_TIENELAVADORA','F_TIENEAUTOMOVIL',
               'F_TIENECOMPUTADOR','E_PAGOMATRICULAPROPIO']

binary_cols = [c for c in binary_cols if c in train.columns]

for c in binary_cols:
    train[c] = map_binary(train[c])
    train[c] = train[c].replace('nan', np.nan)
    train[c] = train[c].map({'SI':1, 'NO':0})
    
    
if 'E_PRIVADO_LIBERTAD' in train.columns:
    counts = train['E_PRIVADO_LIBERTAD'].value_counts(dropna=False)
    # Si casi todo es 'N', eliminar la columna
    if counts.get('N',0) / len(train) > 0.999:
        train = train.drop(columns=['E_PRIVADO_LIBERTAD'])
    else:
        # si es relevante, mapear S->1, N->0
        train['E_PRIVADO_LIBERTAD'] = train['E_PRIVADO_LIBERTAD'].map({'S':1,'N':0})
        
if 'F_ESTRATOVIVIENDA' in train.columns:
    def map_estrato(x):
        x = str(x).strip().upper()
        if 'ESTRATO' in x:
            # extraer número
            parts = [p for p in x.split() if p.isdigit()]
            if parts:
                return int(parts[0])
        if 'SIN' in x:
            return np.nan
        return np.nan
    train['F_ESTRATOVIVIENDA'] = train['F_ESTRATOVIVIENDA'].apply(map_estrato)
        
        
    ## E_HORASSEMANATRABAJA

# Mapear a escala 0–4 (por orden de horas)
horas_map = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}
train['E_HORASSEMANATRABAJA'] = train['E_HORASSEMANATRABAJA'].map(horas_map)


# homogenizar nombres de columnas
train.columns = train.columns.str.strip().str.lower().str.replace('.', '_')


# Eliminar columnas id, periodo

train = train.drop(columns=['id'])

Comparación de F_TIENEINTERNET y F_TIENEINTERNET.1: 692500 iguales de 692500 registros.


In [76]:
train.head(10)

,periodo_academico,e_prgm_academico,e_prgm_departamento,e_valormatriculauniversidad,e_horassemanatrabaja,f_estratovivienda,f_tieneinternet,f_educacionpadre,f_tienelavadora,f_tieneautomovil,e_pagomatriculapropio,f_tienecomputador,f_educacionmadre,rendimiento_global,indicador_1,indicador_2,indicador_3,indicador_4
0,20212,ENFERMERIA,BOGOTÁ,6.0,1.0,3.0,1.0,5.0,1.0,1.0,0.0,1.0,9.0,2,0.322,0.208,0.310,0.267
1,20212,DERECHO,ATLANTICO,4.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0,1.0,5.0,0,0.311,0.215,0.292,0.264
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,4.0,4.0,3.0,1.0,4.0,1.0,0.0,0.0,0.0,4.0,0,0.297,0.214,0.305,0.264
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,5.0,0.0,4.0,1.0,NaN,1.0,0.0,0.0,1.0,4.0,3,0.485,0.172,0.252,0.190
4,20212,PSICOLOGIA,ANTIOQUIA,4.0,3.0,3.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1,0.316,0.232,0.285,0.294
5,20203,MEDICINA VETERINARIA,ANTIOQUIA,7.0,1.0,5.0,1.0,8.0,1.0,1.0,0.0,1.0,4.0,2,0.553,0.142,0.248,0.175
6,20183,INGENIERIA MECANICA,HUILA,4.0,3.0,2.0,1.0,7.0,1.0,1.0,1.0,1.0,6.0,3,0.242,0.299,0.267,0.308
7,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,3.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,1,0.238,0.316,0.286,0.290
8,20212,INGENIERIA INDUSTRIAL,ATLANTICO,6.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,7.0,1,0.147,0.407,0.212,0.328
9,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,4.0,4.0,5.0,1.0,9.0,1.0,1.0,1.0,1.0,9.0,3,0.535,0.122,0.257,0.167


In [84]:
train.to_csv("./data/train_to_colab.csv", index=False)

In [5]:
import pandas as pd
train = pd.read_csv("./data/train_to_colab.csv")

In [79]:
print("\n🔹 Porcentaje de valores nulos por columna:")
print((train.isnull().sum()/len(train))*100)


🔹 Porcentaje de valores nulos por columna:
periodo_academico              0.000000
e_prgm_academico               0.000000
e_prgm_departamento            0.000000
e_valormatriculauniversidad    0.907870
e_horassemanatrabaja           4.455884
f_estratovivienda              5.115668
f_tieneinternet                3.845343
f_educacionpadre               7.075668
f_tienelavadora                5.743394
f_tieneautomovil               6.299350
e_pagomatriculapropio          0.938339
f_tienecomputador              5.502238
f_educacionmadre               4.129819
rendimiento_global             0.000000
indicador_1                    0.000000
indicador_2                    0.000000
indicador_3                    0.000000
indicador_4                    0.000000
dtype: float64
